In [1]:
from sklearn.preprocessing import FunctionTransformer

from reskit.norms import binar_norm, wbysqdist
from reskit.norms import spectral_norm

from reskit.features.degree import degrees 

from sklearn.feature_selection import VarianceThreshold

from sklearn.preprocessing import MinMaxScaler

from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier 
from xgboost import XGBClassifier

from sklearn.model_selection import StratifiedKFold

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

import os
import pandas as pd
import numpy as np

def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

from reskit.core import Transformer, Pipeliner

def orig(x):
    return x

/home/nsdfyd/.venv/venv34/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Функция чтения данных.

In [2]:
def get_autism(path_to_read='Data/dti/', distances=True):
    def get_autism_distances(loc_name):
        with open(loc_name, 'r') as f:
            read_data = f.readlines()

        read_data = pd.DataFrame(
            np.array([np.array(item[:-1].split()).astype(int) for item in read_data]))

        return read_data

    def get_distance_matrix(coords):
        if type(coords) == pd.core.frame.DataFrame:
            coords = coords.values
        elif type(coords) != np.ndarray:
            print('Provide either pandas df or numpy array!')
            return -1

        shape = len(coords)
        dist_matrix = np.zeros((shape, shape))
        del shape
        for i in range(len(coords)):
            for j in range(i + 1, len(coords)):
                dist_matrix[i, j] = np.linalg.norm(coords[i, :] - coords[j, :])
                dist_matrix[j, i] = dist_matrix[i, j]
        return dist_matrix

    target_vector = []  # this will be a target vector (diagnosis)
    matrices = []  # this will be a list of connectomes
    all_files = sorted(os.listdir(path_to_read))
    matrix_files = [
        item for item in all_files if 'DTI_connectivity' in item and 'All' not in item]
    distance_files = [
        item for item in all_files if 'DTI_region_xyz_centers' in item and 'All' not in item]

    # for each file in a sorted (!) list of files:
    for filename in matrix_files:

        A_dataframe = pd.read_csv(
            path_to_read + filename, sep='   ', header=None, engine='python')
        A = A_dataframe.values  # we will use a list of numpy arrays, NOT pandas dataframes
        matrices.append(A)  # append a matrix to our list
        if "ASD" in filename:
            target_vector.append(1)
        elif "TD" in filename:
            target_vector.append(0)
    asd_dict = {}
    asd_dict['X'] = np.array(matrices)
    asd_dict['y'] = np.array(target_vector)
    if distances:
        dist_matrix_list = []
        for item in distance_files:
            # print(item)
            cur_coord = get_autism_distances(path_to_read + item)
            cur_dist_mtx = get_distance_matrix(cur_coord)
            dist_matrix_list += [cur_dist_mtx]

        asd_dict['dist'] = np.array(dist_matrix_list)

    return asd_dict

Объекты кросс валидации для поиска наилучших параметров грид серчем и для валидирования получившейся модели.

In [3]:
grid_cv = StratifiedKFold(n_splits=10,
                          shuffle=True,
                          random_state=0)

eval_cv = StratifiedKFold(n_splits=10,
                          shuffle=True,
                          random_state=1)

Если понадобится выполнить только один пайплайн, то можно сделать следующее. Допустим мы хотим применить к данным бинарную нормировку и получить степени вершин.

In [4]:
data = 'Data/dti/'
data = Transformer(get_autism).fit_transform(data)
data = Transformer(binar_norm).fit_transform(data)
data = Transformer(degrees, collect=['degrees']).fit_transform(data)

get_autism возвращает словарь с полями 'X', 'dist' и 'y'. А каждая функция в пакете binar_norm, degrees и т.д. принимает словарь с этими полями и изменяет эти поля, либо дополняет новые. Например, в случае degrees эта функция добавляет поле 'degrees'. Класс Transformer просто применяет к данным функцию аналогично FunctionTransformer в sklearn, но также он может вернуть тюпл X, y, где X --- выбранное в collect ( сейчас это ['degrees'] (обязательно лист)) поле, y --- значения по полю 'y'. Значение по полю 'y' выбирать не надо, это делается автоматом, также обязательно нужно по этому полю 'y' хранить таргет когда пишете функцию импорта данных, т.е. как в get_autism.

In [5]:
X, y = data

Зададим пайплайн. Пусть это будет селектор, скейлер и классификатор.

In [6]:
steps = [('selector', VarianceThreshold()), ('scaler', MinMaxScaler()), ('classifier', LogisticRegression())] 
pipeline = Pipeline(steps)

Зададим параметры грид серча и сделаем грид серч. Параметры задаются в виде названиеШага\_\_параметрОбъекта.

In [7]:
param_grid = dict(classifier__penalty=['l1', 'l2'])
scoring = 'roc_auc'
grid_clf = GridSearchCV(estimator=pipeline, param_grid=param_grid, scoring=scoring, n_jobs=-1, cv=grid_cv)
grid_clf.fit(X, y)

GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=0, shuffle=True),
       error_score='raise',
       estimator=Pipeline(steps=[('selector', VarianceThreshold(threshold=0.0)), ('scaler', MinMaxScaler(copy=True, feature_range=(0, 1))), ('classifier', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))]),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'classifier__penalty': ['l1', 'l2']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='roc_auc', verbose=0)

Теперь возьмем классификатор с лучшими параметрами, зададим пайплайн с новым классификатором и оценим получившуюся модель.

In [8]:
steps[-1] = steps[-1][0], grid_clf.best_estimator_
pipeline = Pipeline(steps)
scores = cross_val_score(pipeline, X, y, scoring=scoring, cv=eval_cv, n_jobs=-1)
np.mean(scores), np.std(scores)

(0.47566666666666679, 0.17615050383124087)

Все тоже самое делает класс Pipeliner. Сначала создается таблица.

In [9]:
data = [('UCLAsource', Transformer(get_autism))]
normalizer = [('binar', Transformer(binar_norm))]
featurizer = [('degrees', Transformer(degrees, collect=['degrees']))]
selector = [('var_threshold', VarianceThreshold())]
scaler = [('minmax', MinMaxScaler())]
classifier = [('LR', LogisticRegression())]

steps = [('Data', data),
         ('Normalizer', normalizer),
         ('Featurizer', featurizer),
         ('Selector', selector),
         ('Scaler', scaler),
         ('Classifier', classifier)]

param_grid = dict(
    LR=dict(
        penalty=['l1', 'l2']
    )
)

pipe = Pipeliner(steps, eval_cv=eval_cv, grid_cv=grid_cv, param_grid=param_grid)
pipe.plan_table

,Data,Normalizer,Featurizer,Selector,Scaler,Classifier
0,UCLAsource,binar,degrees,var_threshold,minmax,LR


Теперь сделаем грид серч и валидирование.

In [10]:
results = pipe.get_results('Data/dti/', caching_steps=['Data', 'Normalizer', 'Featurizer'], scoring=['roc_auc'])

                                                     Line: 1/1

In [11]:
results

,Data,Normalizer,Featurizer,Selector,Scaler,Classifier,grid_roc_auc_mean,grid_roc_auc_std,grid_roc_auc_best_params,eval_roc_auc_mean,eval_roc_auc_std,eval_roc_auc_scores
0,UCLAsource,binar,degrees,var_threshold,minmax,LR,0.471099,0.171803,{'penalty': 'l2'},0.475667,0.176151,[ 0.46666667 0.6 0.24 0.9 ...


И если мы хотим попробовать много нормировок, признаков, классификаторов и т.д. То можно сделать следующее.

Задаем колонки и забаненные комбинации. Строк с одновременно встречающимися ключами в итоговой таблице не будет.

In [4]:
data = [('UCLAsource', Transformer(get_autism))]

weighters = [('origW', Transformer(orig)),
             ('binar', Transformer(binar_norm)),
             ('wbysqdist', Transformer(wbysqdist))]

normalizers = [('origN', Transformer(orig)),
               ('spectral', Transformer(spectral_norm))]

featurizers = [('origF', Transformer(orig, collect=['X'])),
               ('degrees', Transformer(degrees, collect=['degrees']))]

selectors = [('var_threshold', VarianceThreshold())]

scalers = [('minmax', MinMaxScaler()),
           ('origS', FunctionTransformer(orig))]

classifiers = [('LR', LogisticRegression()),
               ('RF', RandomForestClassifier()),
               ('SVC', SVC()),
               ('XGB', XGBClassifier(nthread=1)),
               ('SGD', SGDClassifier())]

steps = [('Data', data),
         ('Weighters', weighters),
         ('Normalizers', normalizers),
         ('Featurizers', featurizers),
         ('Selectors', selectors),
         ('Scalers', scalers),
         ('Classifiers', classifiers)]

banned_combos = [('UCLAsource', 'origN'),
                 ('UCLAsource', 'origF'),
                 ('UCLAbaseline', 'degrees'),
                 ('UCLAbaseline', 'binar'),
                 ('UCLAbaseline', 'wbysqdist'),
                 ('UCLAbaseline', 'spectral'),
                 ('LR', 'origS'),
                 ('SVC', 'origS'),
                 ('SGD', 'origS'),
                 ('RF', 'minmax'),
                 ('XGB', 'minmax')]

Задаем параметры грид серча и смотрим, какая таблица получилась. Если хотите вопроизвести результаты из статьи PRNI, то все параметры нужно раскомментировать, однако это считается ДОЛГО.

In [5]:
param_grid = dict(
    LR=dict(
#        C=[0.01, 0.05, 0.1] + [0.05*i for i in range(3, 21)],
#        max_iter=[50, 100, 500],
        penalty=['l1', 'l2']
    ),
    SGD=dict(
#        alpha=[0.001, 0.01, 0.1, 0.5, 1.0],
#        l1_ratio=[0, 0.2, 0.4, 0.6, 0.8, 1],
#        loss=['hinge', 'log', 'modified_huber'],
        n_iter=[50, 100, 200],
#        penalty=['elasticnet']
    ),
    SVC=dict(
#        C=[0.0005, 0.001, 0.005, 0.01] + [i*0.05 for i in range(1,11)],
#        degree=[2, 3, 4],
#        kernel=['linear', 'poly', 'rbf', 'sigmoid'],
        max_iter=[50, 100, 150],
    ),
    RF=dict(
#        criterion=['entropy', 'gini'],
#        max_depth=[3, 5, 7, 10, 20],
#        max_features=['log2', 'sqrt'] + [0.001, 0.005, 0.01, 0.05, 0.1, 0.25, 0.5, 1.0],
        n_estimators=[10, 50, 100, 200, 500]
    ),
    XGB=dict(
#        colsample_bytree=[0.05*i for i in range(1,21)],
#        learning_rate=[0.01*i for i in range(1,6)] + [0.05*i for i in range(2,11)],
#        max_depth=[i for i in range(1,12)],
#        n_estimators=[10, 50, 100, 200, 500],
#        nthread=[1],
#        reg_alpha=[0, 1],
#        reg_lambda=[0, 1],
        subsample=[0.5, 0.7, 1]
    )
)
pipe = Pipeliner(steps, eval_cv=eval_cv, grid_cv=grid_cv, param_grid=param_grid, banned_combos=banned_combos)
pipe.plan_table

,Data,Weighters,Normalizers,Featurizers,Selectors,Scalers,Classifiers
0,UCLAsource,origW,spectral,degrees,var_threshold,minmax,LR
1,UCLAsource,origW,spectral,degrees,var_threshold,minmax,SVC
2,UCLAsource,origW,spectral,degrees,var_threshold,minmax,SGD
3,UCLAsource,origW,spectral,degrees,var_threshold,origS,RF
4,UCLAsource,origW,spectral,degrees,var_threshold,origS,XGB
5,UCLAsource,binar,spectral,degrees,var_threshold,minmax,LR
6,UCLAsource,binar,spectral,degrees,var_threshold,minmax,SVC
7,UCLAsource,binar,spectral,degrees,var_threshold,minmax,SGD
8,UCLAsource,binar,spectral,degrees,var_threshold,origS,RF
9,UCLAsource,binar,spectral,degrees,var_threshold,origS,XGB


Получаем результаты.

In [ ]:
results = pipe.get_results('Data/dti/', caching_steps=['Data', 'Weighters', 'Normalizers', 'Featurizers'], scoring=['roc_auc'])

Line: 1/15
Line: 2/15
Line: 3/15
Line: 4/15
Line: 5/15
Line: 6/15
Line: 7/15
Line: 8/15
Line: 9/15
Line: 10/15
Line: 11/15


/home/nsdfyd/.venv/venv34/lib/python3.5/site-packages/reskit/norms.py:133: RuntimeWarning: invalid value encountered in true_divide
  weighted_X = X / dist ** 2


Line: 12/15
Line: 13/15


In [ ]:
results